In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import warnings
warnings.filterwarnings('ignore')

import altair as alt

from pandas_profiling import ProfileReport

# Load data for each grade

In [2]:
df_4 = pd.read_csv('grade_4_hdr.csv')

In [3]:
df_8 = pd.read_csv('grade_8_hdr.csv')

In [4]:
df_4 = df_4.drop(columns=['country_ID'])

In [5]:
df_4 = df_4.drop_duplicates(subset=['IDSTUD'])

In [6]:
df_8 = df_8.drop(columns=['country_ID'])

In [7]:
df_8 = df_8.drop_duplicates(subset=['IDSTUD'])

# Missing values identification

In [8]:
def missing_per_df(df):
    
    # Caculate percentage of missing value per column
    percent_missing = df.isnull().sum() * 100 / len(df)
    
    # Create a dataframe
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                     'percent_missing': percent_missing}).reset_index().drop(columns='index')
    # Sort dataframe by percent missing
    missing_value_df.sort_values('percent_missing', inplace=True, ascending=False)
    
    return missing_value_df.head(10)

In [9]:
missing_per_df(df_4)

,column_name,percent_missing
89,Secondary schools with access to the Internet,50.758354
88,Primary schools with access to the Internet,50.758354
68,Youth literacy rate Female,38.316360
69,Youth literacy rate Male,38.316360
77,Government expenditure on education,36.271762
95,Share of population with safe drinking water,25.525554
75,Primary school dropout rate,25.422779
76,Lower secondary retention,24.164564
71,Pre-primary,21.797627
74,Tertiary,21.757140


# Missing values manipulation

In [10]:
def fillna(df):
    
    # Columns to ignore when filling na
    id_cols = ['IDCNTRY', 'IDSCHOOL', 'IDCLASS', 'IDSTUD', 'Country', 'grade_4', 'grade_8']

    # Columns to fill na
    cols = [col for col in df.columns if col not in id_cols]

    # Columns to group by
    groupby_cols = ['IDCNTRY', 'IDSCHOOL', 'IDCLASS']
    
    # Fillna by three layers of identifications
    for i in range(3, 0, -1):
        group_cols = groupby_cols[0:i]
        for col in cols:
            df[col] = df.groupby(group_cols)[col].apply(lambda x: x.fillna(x.median()))
    
    # Fillna of the residuals with new category
    df = df.fillna('No Information')
    
    return df

In [11]:
# Fillna for both grades
df_4 = fillna(df_4)
df_8 = fillna(df_8)

In [12]:
# # Export to csv
# df_4.to_csv('grade_4_final.csv', index=False)
# df_8.to_csv('grade_8_final.csv', index=False)

In [13]:
# Create profile reports
# profile4 = ProfileReport(df_4, title='Grade 4 Report')
# profile4.to_file("grade_4_report.html")
# profile8 = ProfileReport(df_8, title='Grade 8 Report')
# profile8.to_file("grade_8_report.html")